In [2]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.channels.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import numpy as np
import isodate
import pickle
import datetime
import pytz
from dateutil import tz
from IPython.display import JSON
from datetime import timedelta

pd.set_option('display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

 # Cleaning up the dataset(s)

In [3]:
df= pd.read_excel("Hololive_channel_video_data_REF.xlsx", index_col=None)
df_2022 = pd.read_excel('output_2022_REF.xlsx', index_col=None)
df_stat = pd.read_excel('Hololive_channel_stats.xlsx', index_col=None)

In [4]:
df_u = pd.merge(df_2022, df, how = 'left', left_on = 'Video_ID', right_on = 'video_id')

In [5]:
df_u = df_u.drop_duplicates(subset = ['Video_ID'])
df_u.shape

(13439, 30)

In [6]:
df_sp_ult = pd.merge(df_u, df_stat, how = 'left', left_on = 'Creator', right_on = 'Creator')
df_sp_ult = df_sp_ult.drop_duplicates(subset = ['Video_ID'])
df_sp_ult.shape

(13439, 36)

In [9]:
df_sp_ult = df_sp_ult.drop(columns=['video_id','title', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount', 'definition', 'caption', 'views', 'playlistId'])
#df_sp_ult[df_sp_ult['Creator'] == 'Sakamata Chloe']

In [10]:
df_sp_ult.shape

(13439, 26)

In [11]:
tmp = df_sp_ult[df_sp_ult['Branch'].isna()]
tmp['Creator'].unique()

array(['Ayamy', 'Hololive Official', 'Natsuiro Matsuri', 'Shigure Ui',
       'Kagura Nana', 'Pochimaru', 'Hololive Indonesia Official',
       'Aoi Nabi', 'Hololive English Official', 'Uruha Rushia'],
      dtype=object)

In [12]:
tmp = df_sp_ult.drop(df_sp_ult[df_sp_ult.Creator.isin(['Ayamy', 'Hololive Official', 'Shigure Ui','Kagura Nana', 'Pochimaru', 'Hololive Indonesia Official','Aoi Nabi', 'Hololive English Official'])].index)
tmp.loc[tmp['Creator'] == 'Uruha Rushia', 'Branch'] = 'Japan'
tmp.loc[tmp['Creator'] == 'Natsuiro Matsuri', 'Branch'] = 'Japan'
tmp['Creator'].isna().any()

False

In [13]:
tmp.shape

(12826, 26)

### Looking at the Data, A large selection of the $0 superchat streams are from membership streams and The Game Elden Ring for some reason, so we are keeping 0s as 0s

In [15]:
pd.set_option('display.max_rows', None)
tmpna = tmp
tmpna['SuperChat_total'] = tmpna['SuperChat_total'].replace(0.00, np.nan)
#tmpna = tmp[tmp['duration'].isna()]
#tmpna['Creator'].value_counts()
#tmp[tmp['SuperChat_total'].isna()]
tmpna = tmpna[tmpna['SuperChat_total'].isna()]

# Percentage of Vids with $0 in SuperChats
((tmpna['Creator'].value_counts()/tmp['Creator'].value_counts())*100).sort_values(ascending = False)

Oozora Subaru        36.363636
Akai Haato           28.627451
Mori Calliope        24.122807
Watson Amelia        23.107570
Ninomae Ina'nis      22.689076
Amane Kanata         18.316832
Ouro Kronii          16.666667
Tokino Sora          15.418502
Shirogane Noel       14.553991
IRyS                 14.285714
Airani Iofifteen     13.402062
Gawr Gura            13.333333
Kazama Iroha         13.191489
La+ Darknesss        13.043478
Shishiro Botan       12.556054
Sakamata Chloe       12.350598
Takanashi Kiara      12.307692
Yukihana Lamy        12.030075
Shiranui Flare       11.387900
Aki Rosenthal        11.210762
AZKi                 11.111111
Kureiji Ollie        11.111111
Ceres Fauna           9.756098
Nekomata Okayu        9.638554
Usada Pekora          9.121622
Kobo Kanaeru          9.012876
Murasaki Shion        8.955224
Sakura Miko           8.278146
Hoshimachi Suisei     7.936508
Moona Hoshinova       7.623318
Takane Lui            7.565012
Pavolia Reine         7.377049
Omaru Po

In [16]:
pd.set_option('display.max_rows', 25)

In [19]:
#tmp['SuperChat_total'].fillna(tmp.groupby('Creator')['SuperChat_total'].transform('mean'))
#tmp['Avg_viewers'] = tmp['Avg_viewers'].replace('---', np.nan)
#tmp['Max_viewers'] = tmp['Max_viewers'].replace('---', np.nan)

tmp['SuperChat_total'] = tmp['SuperChat_total'].replace(np.nan, 0.00)
tmp['Max_viewers'] = tmp['Max_viewers'].fillna(tmp.groupby('Creator')['Max_viewers'].transform('mean'))
tmp['Avg_viewers'] = tmp['Avg_viewers'].fillna(tmp.groupby('Creator')['Avg_viewers'].transform('mean'))

#pd.set_option('display.max_rows', 25)
#tmp[tmp['Avg_viewers'].isna()]
#tmp[tmp['Max_viewers'].isna()]

### While cleaning I noticed there were some entires that had null values for duration and published time. So I went back and rewrote the collection code to take in duration and start time (UST)

In [ ]:
tmp['duration'].isnull().sum()

In [ ]:
tmp['duration'].isnull().sum()
tmp['publishedAt'].isnull().sum()
null_rows = tmp[tmp['duration'].isnull()]
URL_null = null_rows['URL'].to_numpy()
#pickle.dump(URL_null, open(f'Null Duration URLs.pkl','wb'))

#### Insert Pickle file into Scraper and get back the below file

In [20]:
df_nt = pd.read_excel('output_2022_DURATION_NULL.xlsx')

In [21]:
df_nt.drop(df_nt[df_nt['duration'] == ' NaN:NaN:NaN '].index, inplace = True)

In [22]:
# Convert Duration intos seconds
# Original scraper did not take in the start time and duration so we convert

df_nt['duration'] = df_nt['duration'].str.strip()
df_nt['Stream_Start_Time_UST'] = df_nt['Stream_Start_Time_UST'].str.strip()

for x in range(0, len(df_nt)):
    if len(df_nt['duration'].iloc[x]) <= 5:
        df_nt['duration'].iloc[x] = '00:'+df_nt['duration'].iloc[x]

for x in range(0, len(df_nt)):
    
    hms = df_nt['duration'].iloc[x].split(':')
    seconds = int(hms[0])*3600+int(hms[1])*60+int(hms[2])
    df_nt['duration'].iloc[x] = seconds

df_nt.head()

C:\Users\Zach\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Unnamed: 0,URL,Video_ID,Video_Title,Creator,Avg_viewers,Max_viewers,SuperChat_total,Stream_Start_Time_UST,duration
0,0,https://holo.poi.cat/stream/iSZfA1X-lzE,iSZfA1X-lzE,【UNARCHIVED KARAOKE】 Singing til the new year!!!,Ceres Fauna,"13,542","16,155","$2,939.12",05:53:07 - 08:07:32,8065
1,1,https://holo.poi.cat/stream/iFv7Bf53Do8,iFv7Bf53Do8,【NYE】LET'S CELEBRATE! #2023,Watson Amelia,"11,736","19,255","$3,337.57",03:01:04 - 05:24:25,8601
2,2,https://holo.poi.cat/stream/1U8sZXsNUFw,1U8sZXsNUFw,【ポケモンSV】色違いイーブイ孵化チャレンジ🔥今年最後に運はくるのか！孵化しおわったら交換会...,Kazama Iroha,"8,724","10,382","$3,256.51",05:05:29 - 09:11:07,14738
3,3,https://holo.poi.cat/stream/BEr0tG1g4h8,BEr0tG1g4h8,【歌枠】悪魔の保健医の今年最後のバーストストリーム【ホロライブ/癒月ちょこ】,Yuzuki Choco,"3,064","3,948",$33.42,16:49:33 - 18:18:43,5350
4,4,https://holo.poi.cat/stream/pyHtS18BamQ,pyHtS18BamQ,【MoonUtau】Unarchive home3D sudden Karaoke【Unar...,Moona Hoshinova,"5,432","6,533",$46.70,15:18:08 - 17:08:38,6630


In [131]:
tmp2 = tmp

In [132]:
tmp2 = tmp2.set_index('Video_ID')
tmp2 = tmp2.fillna(df_nt.set_index('Video_ID'))
tmp2['duration'] = tmp2['duration'].fillna(0)

In [133]:
tmp2 = tmp2.merge(df_nt[['Video_ID', 'Stream_Start_Time_UST']], on = 'Video_ID' , how = 'left')

In [134]:
tmp2['Stream_Start_Time_UST'].notnull().sum()

741

In [135]:
tmp2 = tmp2.drop(1688)
tmp2 = tmp2.reset_index()

In [136]:
pd.set_option('display.max_rows', 750)
tmp2 = tmp2.sort_values(by='Stream Start Time UST', na_position='first', ascending = False)
tmp3 = tmp2.iloc[:741]
tmp2 = tmp2[741:]

In [1]:
pd.set_option('display.max_rows', 750)

# Percentage of superchats with missing dates
((tmp3.groupby('Creator')['SuperChat_total'].sum()/tmp2.groupby('Creator')['SuperChat_total'].sum())*100).sort_values(ascending = False)

NameError: name 'pd' is not defined

In [159]:
# Pandas has no function that allows you to insert in between rows. All of the solutions I found made you reset the index and only really worked for a singl instance. 
# We dont want to reset the indexs as they are imperative to keeping the chronilogical order of our dataset

ins_list = tmp3.values.tolist()
og_list = tmp2.values.tolist()

for x in range(len(ins_list)):
    og_list.insert(ins_list[x][0], ins_list[x])
    
og_list.insert(0, list(tmp2.columns.values))

df = pd.DataFrame(og_list)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

2224


In [171]:
x = 9
ins_list[x][0]
ins_list[x][3]
ins_list[x][4]

150

'【 Fall Guys 】久しぶりにプレイしたら絶叫とお口悪ーナになった！なのら【姫森ルーナ/ホロライブ】'

'Himemori Luna'

In [ ]:
pd.set_option('display.max_rows', None)
#df[df['Stream Start Time UST'].isna()]
#df[df['Video_ID'] == 'jmD8lgQQBDc']
#df.loc[12000:12050]

In [181]:
%%capture
def convert(first_val_index):
    
    hms = df['Stream Start Time UST'].iloc[first_val_index].strftime("%H:%M:%S").split(':')
            
    start = df['Stream_Start_Time_UST'].iloc[x].split('-')[0].strip()
    start = start.split(':')

    end = df['Stream_Start_Time_UST'].iloc[x].split('-')[1].strip()
    end = end.split(':')

    hours = int(hms[0])
    minutes = int(hms[1])
    seconds = int(hms[2])

    start_h = int(start[0])
    start_m = int(start[1])
    start_s = int(start[2])

    end_h = int(end[0])
    end_m = int(end[1])
    end_s = int(end[2])


    insert_into = df['Stream Start Time UST'].iloc[first_val_index] - timedelta(hours=hours, minutes=minutes, seconds=seconds)
    published = insert_into + timedelta(hours=end_h, minutes=end_m, seconds=end_s)

    if (insert_into.year == 2023): 
        insert_into = insert_into - timedelta(days=1)

    insert_into = insert_into + timedelta(hours=start_h, minutes=start_m, seconds=start_s)


    df['publishedAt'].iloc[x] = published

    df['Stream Start Time UST'].iloc[x] = insert_into
    df['Month UST'].iloc[x] = df['Stream Start Time UST'].iloc[x].month_name()
    df['Day of Week UST'].iloc[x] = df['Stream Start Time UST'].iloc[x].day_name()

    df['Stream Start Time JST'].iloc[x] = insert_into.tz_localize('Japan')
    df['Month JST'].iloc[x] = df['Stream Start Time JST'].iloc[x].month_name()
    df['Day of Week JST'].iloc[x] = df['Stream Start Time JST'].iloc[x].day_name()

    df['Stream Start Time CST'].iloc[x] = insert_into.tz_localize('US/Central')
    df['Month CST'].iloc[x] = df['Stream Start Time CST'].iloc[x].month_name()
    df['Day of Week CST'].iloc[x] = df['Stream Start Time CST'].iloc[x].day_name()

    df['Stream Start Time PST'].iloc[x] = insert_into.tz_localize('US/Pacific')

In [182]:
%%capture
for x in range(0, len(df)):
    # df['publishedAt'] = df['publishedAt'].fillna(df2.groupby('Creator')['transit_stations'].transform('mean'))
    
    if (pd.isnull(df['Stream Start Time UST'].iloc[x])) & (df['duration'].iloc[x] > 0.0):
        try:
            first_val_index = df['Stream Start Time UST'].iloc[x:].first_valid_index()
            convert(first_val_index)
            if abs(x - int(first_val_index)) > 25:
                print(x, first_val_index)
        except:   
            print('go fuck yourself')
            first_val_index = df['Stream Start Time UST'].iloc[:x].first_valid_index()
            convert(first_val_index)

In [187]:
df[df['Stream Start Time UST'].isna()]
df[df['publishedAt'].isna()]

,index,Video_ID,URL,Video_Title,Creator,Avg_viewers,Max_viewers,SuperChat_total,channelTitle,description,tags,publishedAt,Month UST,Day of Week UST,Stream Start Time UST,Month JST,Day of Week JST,Stream Start Time JST,Month CST,Day of Week CST,Stream Start Time CST,Stream Start Time PST,duration,channelName,Branch,subscribers,totalVideos,Stream_Start_Time_UST


,index,Video_ID,URL,Video_Title,Creator,Avg_viewers,Max_viewers,SuperChat_total,channelTitle,description,tags,publishedAt,Month UST,Day of Week UST,Stream Start Time UST,Month JST,Day of Week JST,Stream Start Time JST,Month CST,Day of Week CST,Stream Start Time CST,Stream Start Time PST,duration,channelName,Branch,subscribers,totalVideos,Stream_Start_Time_UST


In [188]:
df.iloc[279]

0
index                                                                  279
Video_ID                                                       1H0Sngat8Hw
URL                                https://holo.poi.cat/stream/1H0Sngat8Hw
Video_Title              ≪CHRISTMAS KARAOKE≫ preface. this is not gonna...
Creator                                                        Hakos Baelz
Avg_viewers                                                         5322.0
Max_viewers                                                         8250.0
SuperChat_total                                                     1090.9
channelTitle                                                           NaN
description                                                            NaN
tags                                                                   NaN
publishedAt                                            2022-12-24 04:29:59
Month UST                                                         December
Day of Week UST        

In [189]:
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc = True).dt.tz_convert(None)
df['Stream Start Time CST'] = pd.to_datetime(df['Stream Start Time CST'], utc = True).dt.tz_convert('US/Central')
df['Stream Start Time JST'] = pd.to_datetime(df['Stream Start Time JST'], utc = True).dt.tz_convert('Japan')
df['Stream Start Time PST'] = pd.to_datetime(df['Stream Start Time PST'], utc = True).dt.tz_convert('US/Pacific')
df['Stream Start Time UST'] = pd.to_datetime(df['Stream Start Time UST'], utc = True)

In [190]:
df.shape

(12825, 28)

In [191]:
pd.set_option('display.max_rows', 25)
df = df.loc[~((df['Stream Start Time JST'].dt.year == 2023) & (df['Branch'] == 'Japan'))]
df = df.loc[~((df['Stream Start Time JST'].dt.year == 2023) & (df['Branch'] == 'Indonesia'))]
df = df.loc[~((df['Stream Start Time PST'].dt.year == 2023) & (df['Branch'] == 'English'))]
df.shape

(12791, 28)

In [192]:
pd.set_option('display.max_rows', None)
df.isna().sum()

0
index                        0
Video_ID                     0
URL                          0
Video_Title                  0
Creator                      0
Avg_viewers                  0
Max_viewers                  0
SuperChat_total              0
channelTitle               741
description                747
tags                      3901
publishedAt                  0
Month UST                    0
Day of Week UST              0
Stream Start Time UST        0
Month JST                    0
Day of Week JST              0
Stream Start Time JST        0
Month CST                    0
Day of Week CST              0
Stream Start Time CST        0
Stream Start Time PST        0
duration                     0
channelName                278
Branch                       0
subscribers                278
totalVideos                278
Stream_Start_Time_UST    12050
dtype: int64

In [ ]:
pd.set_option('display.max_rows', 25)

In [ ]:
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc = True).dt.tz_convert(None)
df['Stream Start Time CST'] = pd.to_datetime(df['Stream Start Time CST']).dt.tz_convert(None)
df['Stream Start Time JST'] = pd.to_datetime(df['Stream Start Time JST']).dt.tz_convert(None)
df['Stream Start Time PST'] = pd.to_datetime(df['Stream Start Time PST']).dt.tz_convert(None)
df['Stream Start Time UST'] = pd.to_datetime(df['Stream Start Time UST']).dt.tz_convert(None)

In [ ]:
df.iloc[0:10]

In [202]:
#df[df['Stream Start Time UST'].dt.year == 2023]
df.at[5, 'Stream Start Time UST'] = df.at[5, 'Stream Start Time UST'] - timedelta(days=1)

In [203]:
df.to_excel('Hololive_ULT_2022.xlsx')
#Holostats = pd.read_excel('Hololive_ULT_2022.xlsx')